In [1]:
import optuna
import tensorflow as tf
from sklearn.metrics import f1_score
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [3]:
import pandas as pd

In [4]:
train_df = pd.read_csv("train.csv")
train_df

,Unnamed: 0,file_name,label
0,0,train_data/a6dcb93f596a43249135678dfcfc17ea.jpg,1
1,1,train_data/041be3153810433ab146bc97d5af505c.jpg,0
2,2,train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg,1
3,3,train_data/8542fe161d9147be8e835e50c0de39cd.jpg,0
4,4,train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg,1
...,...,...,...
79945,79945,train_data/9283b107f6274279b6f15bbe77c523aa.jpg,0
79946,79946,train_data/4c6b17fe6dd743428a45773135a10508.jpg,1
79947,79947,train_data/1ccbf96d04e342fd9f629ad55466b29e.jpg,0
79948,79948,train_data/ff960b55f296445abb3c5f304b52e104.jpg,1


In [5]:
train_df.label = train_df.label.map({1: 'ai', 0: 'human'})
train_df

,Unnamed: 0,file_name,label
0,0,train_data/a6dcb93f596a43249135678dfcfc17ea.jpg,ai
1,1,train_data/041be3153810433ab146bc97d5af505c.jpg,human
2,2,train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg,ai
3,3,train_data/8542fe161d9147be8e835e50c0de39cd.jpg,human
4,4,train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg,ai
...,...,...,...
79945,79945,train_data/9283b107f6274279b6f15bbe77c523aa.jpg,human
79946,79946,train_data/4c6b17fe6dd743428a45773135a10508.jpg,ai
79947,79947,train_data/1ccbf96d04e342fd9f629ad55466b29e.jpg,human
79948,79948,train_data/ff960b55f296445abb3c5f304b52e104.jpg,ai


In [6]:
train_data = img_gen.flow_from_dataframe(
    train_df,
    x_col='file_name',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    subset="training"
)

Found 63960 validated image filenames belonging to 2 classes.


In [7]:
val_data = img_gen.flow_from_dataframe(
    train_df,
    x_col='file_name',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    subset="validation"
)

Found 15990 validated image filenames belonging to 2 classes.


In [8]:
base_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

base_model.trainable = True


unfreeze_layers = 0  
for layer in base_model.layers[:-unfreeze_layers]:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  
    
    layers.Dense(640, kernel_initializer="he_normal"),
    layers.LeakyReLU(negative_slope=0.1),
    layers.BatchNormalization(),
    layers.Dropout(0.3359853061113047),

    layers.Dense(256, kernel_initializer="he_normal", activation="mish"),
    layers.BatchNormalization(),

    layers.Dense(512, activation='relu', kernel_initializer="he_normal"),

    layers.Dense(1, activation='sigmoid')
])

In [9]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.FalseNegatives(),
    ],
)

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_f1_score', mode='max', save_best_only=True)
]


model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

C:\Users\PYTHON\PycharmProjects\ai_vs_human\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
 260/1999 ━━━━━━━━━━━━━━━━━━━━ 44:52 2s/step - binary_accuracy: 0.8704 - false_negatives: 221.2154 - loss: 0.3090